# Testing models

In [1]:
import sys

sys.path.append('src/')

from Models import deberta_base_nli, bart_nli
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from Helpers import *

# create helper function to standardise order


In [2]:
deberta_base_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

TypeError: unhashable type: 'list'

In [3]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

([['0.93711895', '0.05809933', '0.00478182']], ['contradiction'])

In [9]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'What is my favourite city?')

([['0.93711895', '0.05809933', '0.00478182']], ['contradiction'])

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def deberta_v3_nli(premise, hypothesis):
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

  model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSequenceClassification.from_pretrained(model_name)


  input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
  output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
  print(output)
  prediction = torch.softmax(output["logits"][0], -1).tolist()
  label_names = ["entailment", "neutral", "contradiction"]
  prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
  print(prediction)



In [2]:

premise = "Angela Merkel ist eine Politikerin in Deutschland und Vorsitzende der CDU"
hypothesis = "Emmanuel Macron is the President of France"

deberta_v3_nli(premise, hypothesis)

/usr/local/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


SequenceClassifierOutput(loss=None, logits=tensor([[-3.2874, -0.8842,  3.8712]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
{'entailment': 0.1, 'neutral': 0.9, 'contradiction': 99.1}
